<a href="https://colab.research.google.com/github/CyrilleMesue/dnagenes.github.io/blob/main/Processing_Dicom_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pydicom


     |████████████████████████████████| 1.9MB 3.9MB/s 


In [4]:
!pip install pylibjpeg pylibjpeg-libjpeg pydicom

     |████████████████████████████████| 4.2MB 4.7MB/s 
     |████████████████████████████████| 3.6MB 38.4MB/s 
     |████████████████████████████████| 890kB 38.9MB/s 


In [5]:
!pip uninstall numpy
!pip install --upgrade numpy

Uninstalling numpy-1.19.5:
  Would remove:
    /usr/bin/f2py
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/dist-packages/numpy-1.19.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libopenblasp-r0-09e95953.3.13.so
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.19.5
     |████████████████████████████████| 15.3MB 62.8MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have

In [1]:
import numpy as np
import pydicom
import pylibjpeg
import os
import matplotlib.pyplot as plt
import cv2
import dask
from tqdm.notebook import tqdm
from dask.diagnostics import ProgressBar
from pathlib import Path

In [7]:
ischemic_dicom_images = Path("/content/drive/MyDrive/BrainCT/TRAINING/KANAMA/png/")
ischemic_dicom_img_paths = sorted(list(ischemic_dicom_images.glob("*.png")))
len(ischemic_dicom_img_paths)

89

In [8]:
ischemic_dicom_images = Path("/content/drive/MyDrive/BrainCT/TRAINING/ISKEMI/DICOM/")
ischemic_dicom_img_paths = sorted(list(ischemic_dicom_images.glob("*.dcm")))
hemorriage_dicom_images = Path("/content/drive/MyDrive/BrainCT/TRAINING/KANAMA/DICOM/")
hemorriage_dicom_img_paths = sorted(list(hemorriage_dicom_images.glob("*.dcm")))

In [9]:
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps

In [10]:
def Dicom_to_Image(Path):
    dcm_Img = pydicom.read_file(Path)

    rows = dcm_Img.get(0x00280010).value #Get number of rows from tag (0028, 0010)
    cols = dcm_Img.get(0x00280011).value #Get number of cols from tag (0028, 0011)

    #Instance_Number = int(dcm_Img.get(0x00200013).value) #Get actual slice instance number from tag (0020, 0013)

    #Window_Center = dcm_Img.get(0x00281050).value #Get window center from tag (0028, 1050)
    #Window_Width = dcm_Img.get(0x00281051).value #Get window width from tag (0028, 1051)
    Window_Center = np.array(dcm_Img.WindowCenter)
    Window_Width = np.array(dcm_Img.WindowWidth)
    
    if Window_Center.size > 1:
      Window_Max = Window_Center[0] + Window_Width[0] / 2
      Window_Min = Window_Center[0] - Window_Width[0] / 2
    else:
      Window_Max = Window_Center + Window_Width / 2
      Window_Min = Window_Center - Window_Width / 2


    if (dcm_Img.get(0x00281052) is None):
        Rescale_Intercept = 0
    else:
        Rescale_Intercept = int(dcm_Img.get(0x00281052).value)

    if (dcm_Img.get(0x00281053) is None):
        Rescale_Slope = 1
    else:
        Rescale_Slope = int(dcm_Img.get(0x00281053).value)

    New_Img = np.zeros((rows, cols), np.uint8)
    Pixels = dcm_Img.pixel_array

    for i in range(0, rows):
        for j in range(0, cols):
            Pix_Val = Pixels[i][j]
            Rescale_Pix_Val = Pix_Val * Rescale_Slope + Rescale_Intercept

            if (Rescale_Pix_Val > Window_Max): #if intensity is greater than max window
                New_Img[i][j] = 255
            elif (Rescale_Pix_Val < Window_Min): #if intensity is less than min window
                New_Img[i][j] = 0
            else:
               New_Img[i][j] = int(((Rescale_Pix_Val - Window_Min) / (Window_Max - Window_Min)) * 255) #Normalize the intensities

    

    return New_Img #, Instance_Number

In [11]:
def dcm_to_png(filepath):
  image = Dicom_to_Image(filepath)
  dcm_filename = filepath.parts[-1]
  png_filename = dcm_filename.replace('.dcm','.png')
  cv2.imwrite(outdir + png_filename, image)

In [12]:
outdir = '/content/drive/MyDrive/BrainCT/TRAINING/ISKEMI/png/'
if not os.path.exists(outdir):
  os.mkdir(outdir)

for imagepath in tqdm(ischemic_dicom_img_paths):
  dcm_to_png(imagepath)

/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:177: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "


In [13]:
outdir = '/content/drive/MyDrive/BrainCT/TRAINING/KANAMA/png/'
if not os.path.exists(outdir):
  os.mkdir(outdir)
for imagepath in tqdm(hemorriage_dicom_img_paths):
  dcm_to_png(imagepath)

/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:177: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "
/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:177: UserWarning: The (0028,0101) 'Bits Stored' value (13-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "
